# Start Interactive Dask Cluster

In [1]:
pip install --upgrade dask-cloudprovider

 skipping upgrade: azureml-sdk>=1.0.83 in /Users/cody/miniconda3/envs/dkdc/lib/python3.8/site-packages (from dask-cloudprovider) (1.14.0)
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install --upgrade azureml-sdk

Requirement already up-to-date: azureml-sdk in /Users/cody/miniconda3/envs/dkdc/lib/python3.8/site-packages (1.14.0)
Note: you may need to restart the kernel to use updated packages.


**RESTART YOUR KERNEL**

## Workspace

Get the Azure ML workspace.

In [3]:
from azureml.core import Workspace

ws = Workspace.from_config()
ws

Workspace.create(name='AzureML', subscription_id='6560575d-fa06-4e7d-95fb-f962e74efd7a', resource_group='cody-eastus-rg')

### Start cluster

In [4]:
from dask_cloudprovider import AzureMLCluster

vm_size = 'STANDARD_DS13_V2'
timeout = 7200 

cluster = AzureMLCluster(ws, vm_size=vm_size, scheduler_idle_timeout=timeout, jupyter=True, show_output=True)

################## Creating new compute targe: dask-ct-369924 ##################
Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
############################## Setting up cluster ##############################
WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.
WARNING - If 'arguments' has been provided here and arguments have been specified in 'run_config', 'arguments' provided in ScriptRunConfig initialization will take precedence.
####################### Waiting for scheduler node's IP ########################
............................................

########################### Scheduler: 10.0.0.4:8786 ###########################
############################# Not on the same VNET #############################
###################### Running in compute instance? False #####################

In [5]:
cluster.run

Experiment,Id,Type,Status,Details Page,Docs Page
dask-cloudprovider,dask-cloudprovider_1601064317_3786ad79,azureml.scriptrun,Running,Link to Azure Machine Learning studio,Link to Documentation


In [6]:
cluster.scale(20) # need more than default quota for this 

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


In [7]:
# print out cluster widget 
cluster

In [9]:
from dask.distributed import Client 

c = Client(cluster)
c

Client Scheduler: tcp://localhost:9002 Dashboard: http://localhost:9001,Cluster Workers: 1 Cores: 8 Memory: 59.10 GB


In [7]:
#cluster.close()

In [10]:
!pip install --upgrade git+https://github.com/dask/adlfs fsspec

  Cloning https://github.com/dask/adlfs to /private/var/folders/gk/kpsykczx509cvpggyml79rsc0000gn/T/pip-req-build-u5gtw6t_
Requirement already up-to-date: fsspec in /Users/cody/miniconda3/envs/dkdc/lib/python3.8/site-packages (0.8.3)
  Created wheel for adlfs: filename=adlfs-0.5.3+18.g65de4a6-py3-none-any.whl size=18432 sha256=cdf1e52bf22b207df145fdd4a4d0cde415c3ded9600c7c923cfdf408e6ccc70a
  Stored in directory: /private/var/folders/gk/kpsykczx509cvpggyml79rsc0000gn/T/pip-ephem-wheel-cache-ipq4em9r/wheels/a1/6c/2d/3b988aa0fd36594a0291addc195e10386681f4f354c95fbe75
Successfully built adlfs
  Attempting uninstall: adlfs
    Found existing installation: adlfs 0.5.3+18.g65de4a6
    Uninstalling adlfs-0.5.3+18.g65de4a6:
      Successfully uninstalled adlfs-0.5.3+18.g65de4a6


In [ ]:
def update_packages():
    import os
    os.system('pip install --upgrade git+https://github.com/dask/adlfs fsspec')

In [ ]:
import dask 

to_compute = [dask.delayed(update_packages) for i in range(20)]
c.compute(to_compute)

In [11]:
from adlfs import AzureBlobFileSystem

storage_options = {'account_name': 'azureopendatastorage'}

fs = AzureBlobFileSystem(**storage_options)
fs.ls('isdweatherdatacontainer')

['isdweatherdatacontainer/ISDWeather/',
 'isdweatherdatacontainer/ISDWeatherAll',
 'isdweatherdatacontainer/ISDWeatherAll/',
 'isdweatherdatacontainer/ISDWeatherBackup',
 'isdweatherdatacontainer/ISDWeatherBackup/',
 'isdweatherdatacontainer/ISDWeatherBackup_20200115/',
 'isdweatherdatacontainer/ISDWeatherTest/',
 'isdweatherdatacontainer/fama',
 'isdweatherdatacontainer/fama/']

In [12]:
import dask.dataframe as dd 

df = dd.read_parquet('abfs://isdweatherdatacontainer/ISDWeather/year=2020/month=1/*.parquet', storage_options=storage_options, columns=['temperature'])
df

,temperature
npartitions=8,
,float64
,...
...,...
,...
,...


In [13]:
df.head()

KilledWorker: ("('read-parquet-head-1-5-read-parquet-43970d1e94ff6a7aee43dd9ff880414f', 0)", <Worker 'tcp://10.0.0.4:33987', name: tcp://10.0.0.4:33987, memory: 0, processing: 1>)